In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [2]:
from azureml import Workspace
ws = Workspace(
    workspace_id='3f2b8c9de077435f8db04b53cc7018f0',
    authorization_token='8X11CFr60SDDs1MLR0uTFfh5esDCdUpNti0jHgRK4q00YX0kQe9BYb7FJgY13SuDc7P1WtFykbjTYY0rhet1Kg==',
    endpoint='https://europewest.studioapi.azureml.net'
)
# load the data
ds = ws.datasets['train.csv']
train = ds.to_dataframe()

ds = ws.datasets['test.csv']
test = ds.to_dataframe()

In [3]:
# get result columns from train
age   = train['age']
sex   = train['sex']
# drop result columns
train = train.drop(['age', 'sex'], axis=1)

In [4]:
# count AUC-ROC
# split TRAIN data on train and test
rf = RandomForestClassifier(n_estimators=20)
# fit classifier
rf.fit(train, sex)
# extract men prefiction from `prediction`
sex_prediction = rf.predict(test)

In [5]:
rfr = RandomForestRegressor(n_estimators=50, n_jobs=-1)
rfr.fit(train, age)
age_prediction = rfr.predict(test)

In [7]:
result = pd.DataFrame(test['user_id'])
result = pd.concat([result, pd.DataFrame(sex_prediction, columns=['sex']), 
                            pd.DataFrame(age_prediction, columns=['age'])], axis=1)
result = result.drop_duplicates(['user_id'])
result.to_csv('data/result_local.csv', index=False)